In [1]:
import pandas as pd
import re

# Load the CSV file
data = pd.read_csv("../../../datasets/test.csv")

# Filter to only include item_weight entity
item_weight_data = data[data['entity_name'] == 'height']

# Extract numeric value and unit from entity_value
def split_value(value):
    value = value.strip()
    parts = value.strip().split(' ')
    if len(parts) == 2:
        try: 
            float(parts[0])
        except:
            return None, None
        return float(parts[0]), parts[1]
    return None, None

# item_weight_data[['numeric_value', 'unit']] = item_weight_data['entity_value'].apply(split_value).apply(pd.Series)

# Drop rows with missing values
item_weight_data.dropna(inplace=True)

# Save processed data to a new CSV file
item_weight_data.to_csv('data/test.csv', index=True)

print("Data preprocessing complete. Processed data saved to 'processed_item_weight_data.csv'.")

Data preprocessing complete. Processed data saved to 'processed_item_weight_data.csv'.


/tmp/ipykernel_127113/1687399857.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  item_weight_data.dropna(inplace=True)


In [5]:
from utils_height import download_images
download_images(data['image_link'].tolist(),  data.index.tolist(), "dataset_test")

 63%|██████▎   | 82619/131187 [1:07:51<39:53, 20.29it/s] 


KeyboardInterrupt: 

In [4]:
import os
import logging
import pandas as pd
# from paddleocr import PaddleOCR
import easyocr
from tqdm import tqdm

# logging.getLogger('ppocr').setLevel(logging.ERROR)
logging.getLogger('easyocr').setLevel(logging.ERROR)

image_folder = 'dataset_test/'  # Adjust this to the correct folder containing your images
# ocr = PaddleOCR(use_angle_cls=True, lang='en')  # Initialize PaddleOCR
reader = easyocr.Reader(['en'], gpu=True)  # Initialize EasyOCR

def extract_ocr_text(index):
    image_path = os.path.join(image_folder, f"{index}.jpg")
    if not os.path.exists(image_path):
        return index, ""
    
    try:
        result = reader.readtext(image_path)
        ocr_text = " ".join([text for _, text, _ in result])
    except Exception as e:
        print(f"Error processing image {index}: {e}")
        ocr_text = ""
    
    return index, ocr_text

ModuleNotFoundError: No module named 'easyocr'

In [ ]:
# Assuming `data` is your DataFrame
data = pd.read_csv("test.csv")
indices = data['id'].to_list()
# Process images in smaller batches to avoid memory issues
batch_size = 10  # Adjust this based on your memory constraints
ocr_results = []

for i in tqdm(range(0, len(indices), batch_size), desc="Extracting OCR"):
    batch_indices = indices[i:i + batch_size]
    batch_indices = batch_indices[::2]
    for index in batch_indices:
        batch_results = []
        batch_results.append(extract_ocr_text(index))
        ocr_results.extend(batch_results)
    

# Add the OCR results to the DataFrame
ocr_text_dict = dict(ocr_results)
data['ocr_text'] = data['id'].map(ocr_text_dict)
data = data[data['ocr_text'].notna() & (data['ocr_text'] != "")]
# Save the updated DataFrame with the OCR text to a new CSV
updated_csv_file = 'test_with_ocr.csv'
data.to_csv(updated_csv_file, index=True)

print(f"OCR data has been added to the CSV. Updated file saved as '{updated_csv_file}'.")

In [ ]:
import tensorflow as tf; print(tf.config.list_physical_devices('GPU'))